# Data Wrangling mit Python I

## Einführung

Die Programmiersprache Python hat in den letzten Jahren unglaublich an Beliebtheit gewonnen (Stack Overflow anfragen mit angeben?). Dies liegt zum einen an der Einsteigerfreundlichkeit und zum anderen an den vielen Anwendungsmöglichkeiten. Vorallem im  wissenschaftlichen Bereich ist wird es für die Data-Analysis und das Machine Learning zu einem einsteigerfreundlichen und mächtigen Werkzeug geworden.  
Im Rahmen dieses Kurses soll in diesem Jupyter-Notebook(dazu gleich mehr) grundlegende Begriffe bei der Arbeit mit Pandas (dazu dann später mehr) vorgestellt werden.  Datenaufbereitung - auch Data Wrangling oder Datan Mungling genant - erlernen zu lassen.



## Womit arbeiten wir?

### Jupyter-Notebook

Unser Arbeitsmittel ist ein Jupyter-Notebook (im weiteren nur noch Notebook genannt). Dieses durch Zellen struktuierte Arbeitsoberflöche lässt uns ausführbaren Code mit stylierbaren Textzeilen verbinden. Dadurch können wir unsere Arbeitsschritte nachvollziehbar halten und zudem dokumentieren.

Grundsätzlich lassen sich die Zellen eines Notebooks in drei Typen unterteilen:

1. Text-Zellen zur formatierten Ausgabe von Text
2. Code-Zellen zur Ausführung von Python-Code
3. Output-Zellen zur Ausgabe der Ergebnisse von Code-Zellen

Anders als bei der [WYSIWYG](https://de.wikipedia.org/wiki/WYSIWYG "Klick") Textbearbeitung - welche bei Microsoft Word oder Libre Office Writer verwendet wird - schreiben wir die Formatierung des Textes mit in den  selbst. Dabei kann sowohl das verkürzte [Markdown-Format](https://markdown.de/) oder auch HTML verwendet werden. Genauere Informationen gibt es in der [Jupyter-Notebook Dokumentation](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html) und auf diesem [Cheatsheet](https://www.heise.de/mac-and-i/downloads/65/1/1/6/7/1/0/3/Markdown-CheatSheet-Deutsch.pdf).

Code-Zellen werden mit als Zellen mit grauem Hintergrund dargestellt und können mittels des kleinen Dreiecks (oder der Tastenkombination <kbd>Ctrl + Enter</kbd>) ausgeführt werden. Probiere es mal mit der nächsten Zelle aus!

In [4]:
# Dies ist eine Zelle, in der Code ausgeführt werden kann.

def say_hello_to(name):
    return f"Hello, my Friend {name}!!!"  # https://realpython.com/python-f-strings/

my_name = "Alex"
print(say_hello_to(my_name))

# Die Output-Zelle für diese Code-Zelle 
# erscheint nach der Ausführung gleich unter ihr
print(my_name)
display(my_name)
my_name

Hello, my Friend Alex!!!
Alex


'Alex'

'Alex'

Super oder? Das Ausführen des Codes funktioniert also ähnlich zur Ausführung eines Python-Skript. Die Code-Zellen besitzen jedoch einige Eigenheiten, die wir von einem Python-Skript nicht kennen. Beispielsweise wird wird die Letzte Zeile einer Code-Zelle ausgegeben, was an der Ausgabe `'Alex'` zu sehen ist. Diese Ausgabe lässt sich auch mit der Methode `display()` erreichen und unterscheidet sich von der Methode `print()`.  
Gucken wir uns eine weitere Eigenheit am folgenden Beispiel an...(bitte Code ausführen)

PS: Überlegt vor der Ausführung kurz, was eigentlich passieren sollte, wenn die untere Code als Python-Skript ausgeführt wird.

In [12]:
print(say_hello_to(my_name))

Hello, my Friend Alex!!!


Die Funktion `say_hello_to()` kann mit dem Parameter `my_name` aufgerufen werden, obwohl weder die Funktion `say_hello_to()` noch die Variable `my_name` in der Code-Zelle definiert wurde. Es zeigt sich, dass die Ergebnisse jeder ausgeführten Code-Zelle auch nach ihrer Beending der Notebook-Umgebung zur Verfügung steht. Damit ist die Reihenfolge der Ausführung bei einem Notebook entscheidend und muss beachtet werden!  
Besonders bei der Importierung von Packeten sowie der Wiederverwendung von Funktionen und Variablen-Namen sollte dieses "Gedächtnis" des Notebooks beachtet werden.

PS: Soll das "Gedächtnis" gelöscht werden, muss der Kernel des Notebooks zurückgesetzt werden. Dies kann in der oberen Tool-Leiste durchgeführt werden.

### Python-Pakete

Um beim Programmieren nicht alles selbst programmieren zu müssen, greifen wir auf Erfahrung anderer Programmierer/-innen mittels Pakete zu.
Grundlage für die Datenaufbereitung sind die folgenden 3 Pakete:

1. **Pandas** für vereinfachte Datenmanipulation und -analyse
2. **Numpy** für numerische Berechnungen und der Arbeit mit Matrizen
3. **Matplotlib** für die Visualiserung (Diagramme, Histogramme, ...)

Diese werden zu Begin unseres Notebooks mittels des folgenden Codes importiert...

In [8]:
# die Abkürzungen am besten beibehalten
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Wichtige Konzepte
Im folgenden Werden wir uns genauer mit einigen wichtigen Konzepte der vorgestellten Pakete beschäftigen. 

### Konzept 1. `Serie` von Pandas

Series welche mit einer einzigen (Zeit-)Reihe vergleichbar ist und bestandteil eines Dataframe ist. Im folgenden sehen wir, wie wir Serien erstellen können 

Tipp: am besten jede Serie mit `s_` beginnen um sie klar von anderen Python-Objekten unterscheidbar zu machen. sehr einfach können wir Python-Listen und NumPy-Arrays zu Serien umbauen ...

In [2]:
# Erstellungsmöglichkeiten einer Serie

# mehr Informationen über die Erstellung von Zufallszahlen gibt es hier: https://docs.scipy.org/doc/numpy-1.15.0/reference/routines.random.html oder https://pandas.pydata.org/pandas-docs/version/0.22.0/generated/pandas.Series.sample.html


float_list = list(np.random.random(10)*180)
s_float = pd.Series(float_list)
print(s_float)
# es können auch NumPy-Objekte - Wie das `numpy.array` - genutzt werden
float_array = np.array(np.random.random(10)*180)
s_float_array = pd.Series(float_array)
print(s_float_array)
!!! Display(serie) nicht vergessen
!!! Vielleicht noch Zugriff mit aufnehmen?? 
Um auf einen bestimmten Wert mittels eines zugreifen zu können, müssen - ähnlich zu einer Liste - die eckigen Klammern verwendet werden

0      3.654183
1    139.385421
2     22.924473
3    149.365162
4    119.722332
5    115.306097
6    128.488179
7     18.509560
8    163.925146
9     63.061617
dtype: float64
0    121.241331
1      9.718013
2    122.002946
3     12.145526
4    162.136387
5    151.973022
6     77.696134
7    145.270794
8      8.540165
9      3.835722
dtype: float64


So weit so gut! Jetz bist du dran. Erstelle in der unteren Code-Zelle die geforderten Objekte 

In [3]:
### W1A1: Serien erstellen ###
# Erstelle die folgenden Serien (die Werte müssen dabei nicht zufällig sein!)

# Eine Serie mit 10 Elementen, aus der Menge M = {1,2,3,4,5,6} 
s_int = pd.Series([1,5,6,3,2,1,4,5,2,1])
# Eine Serie mit 10 Vornamen (alles klein geschrieben)
s_string = pd.Series(["alex", "paul", "paula","erika", "jan","karl", "vlad", "franziska", "sabrina", "florian"])
# Eine Serie, in der Zeichenketten, ganze Zahlen und Gleitkommazahlen vorkommen 
s_mix = pd.Series([1,"zwei", 3.0, 4, "fünf", 6.111, "zieben", 8, 9.9999, np.NaN])
print(s_int, s_string, s_mix, sep="\n\n")

0    1
1    5
2    6
3    3
4    2
5    1
6    4
7    5
8    2
9    1
dtype: int64

0         alex
1         paul
2        paula
3        erika
4          jan
5         karl
6         vlad
7    franziska
8      sabrina
9      florian
dtype: object

0         1
1      zwei
2         3
3         4
4      fünf
5     6.111
6    zieben
7         8
8    9.9999
9       NaN
dtype: object


Noch mehr Möglichkeiten Serien zu erstellen gibt es hier:
mehr Infos und Möglichkeiten gibt es hier: https://www.geeksforgeeks.org/creating-a-pandas-series/

Eigenschaften und Methoden von Serien

Betrachten wir uns die Ausgabe unserer Serien fällt uns der `dtype` am unteren Ende jeder Serie auf. Dieser gibt den Typ der in der vorliegenden Serie an, solange dieser Eindeutig ist.
`dtype` stellt dabei eine von vielen Eigenschaften dar, die eine Serie besitzt. durch den `.`(Punkt) können wir auf diese Eigenschaften zugreifen. 


In [5]:
print(s_int.dtype, s_float_array.dtype, s_float.dtype)

int64 float64 float64


Mit diesem Wissen und der [Dokumentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html) können nun die folgenden Aufgaben erledigt werden...

In [6]:
### W1A2: Eigenschaften von Serien ###
# Erledige die folgenden Aufgaben und gib das Ergebnis aus

# Gib die Anzahl an Elementen in s_float zurück
print(s_float.size)
# überprüfe, ob `s_mix` und `s_string` den selben Datentyp besitzen
print(s_mix.dtype == s_string.dtype)
# gib die 5. Stelle der Serie s_string` aus (Achte auf die Klammern!)
print(s_string[4], s_string.iloc[4])
# gibt die Werte von `s_string` als Liste zurück
print(s_string.values)
# setze den index von von `s_int` auf `s_string`
s_int.index = s_string
print(s_int)

10
True
jan jan
[&#39;alex&#39; &#39;paul&#39; &#39;paula&#39; &#39;erika&#39; &#39;jan&#39; &#39;karl&#39; &#39;vlad&#39; &#39;franziska&#39; &#39;sabrina&#39;
 &#39;florian&#39;]
alex         1
paul         5
paula        6
erika        3
jan          2
karl         1
vlad         4
franziska    5
sabrina      2
florian      1
dtype: int64


Neben den Eigenschaften besitzen Serien auch Methoden, welche ebenfalls mittels `.`(Punkt) aufrufbar sind - genau wie Funktionen - immer ein Klammerpaar besitzen. Als Rückgabewert können dabei neue Serien entstehen oder wichtige Werte über die Serie zurückgegeben werden...


  (elche sich sich eher auf Serie als konzentrieren(Größe, Datentyp, Index) besitzen Serien auch Methoden. . Diesearbeiten - im Gegensatz zu den Eigenschaften die sich eher auf - mit den Werten der Serie. Beispielsweise kann mit () Diese MethodenWir sehen dabei folgendes
Diese 
- dtype: Sind alle Objekte einer Serie vom selben Typ wie integer(ganze Zahlen), float(Gleitkommazahlen) erhällt auch die Serie diese Eigenschaft.
- Serien aus gemischten Typen oder aus Zeichenketten erhalten hingegen den dtype `object`.
`dtype` ist eine Eigenschaft, die jede Serie besitzte und von uns auch abgefragt 

In [7]:
# Die Methode `append()` "klebt" zwei Serien hintereinder und erzeugt somit eine dritte.
s_int_float = s_int.append(s_float)
print(s_int_float)

# Die Methode max() gibt den größten Wert einer numerischen Serie zurück
max_s_int_float = s_int_float.max()
print(f"Der Maximalwert von s_float ist {max_s_float}")

alex           1.000000
paul           5.000000
paula          6.000000
erika          3.000000
jan            2.000000
karl           1.000000
vlad           4.000000
franziska      5.000000
sabrina        2.000000
florian        1.000000
0              3.654183
1            139.385421
2             22.924473
3            149.365162
4            119.722332
5            115.306097
6            128.488179
7             18.509560
8            163.925146
9             63.061617
dtype: float64


NameError: name &#39;max_s_float&#39; is not defined

Eine Serie besitzt unzählige Methoden, die in verschiedenen Situationen Anwendung finden. Löse die folgenden Aufgaben um einige genauer kennen zu lernen...

In [19]:
### W1A3 Funktionen von Serien ###

# gibt die Anzahl aller Werte in `s_mix` zurück, die nicht `np.nan` sind
print(s_mix.count())
# überprüfe, ob deine Werte aus `s_string` in ["ben", "marie", "sophia", "jonas"] liegen
searched_names = ["ben", "marie", "sophia", "jonas"]
print(s_string.isin(searched_names))
# entferne das 1. Element von `s_mix` und gib sowohl das Element als auch die neue Serie aus
#print(s_mix.pop(0))
print(s_mix)
# überprüfe, ob `s_float` Werte vom Typ `na.nan` besitzt
print(s_float.isna())
# gib die Summe aller Werte von `s_float` zurück
print(s_float.sum())
# gib alle vorhandenen Elemente von `s_int` zurück
print(s_int.unique())

8
0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
dtype: bool
1      zwei
2         3
3         4
4      fünf
5     6.111
6    zieben
7         8
8    9.9999
9       NaN
dtype: object
0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
dtype: bool
924.3421703382513
[1 5 6 3 2 4]


Besonders interessant ist die Methode `apply()`, welche eine Funktion mit jedem Wert einer Serie durchführt und diesen somit anpassen kann... 

In [50]:
# Der erste Buchstabe jeder unserer Vornamen in `s_string` wird Groß geschrieben, solange der Vorname nicht Paul ist
def capitalize_if_not_paul(value):
    if not value.upper() == "PAUL":
        return value.capitalize()
    else:
        return value

s_cap_without_paul_s_string = s_string.apply(capitalize_if_not_paul)
print(s_cap_without_paul_s_string)

0         Alex
1         paul
2        Paula
3        Erika
4          Jan
5         Karl
6         Vlad
7    Franziska
8      Sabrina
9      Florian
dtype: object


Die Möglichkeit weitere Parameter an die Funktion zu übergeben und zukünftige Arbeitsschritte und Berechnungen vorzubereiten; macht `apply()` zu einem mächtigen Werzeug...

In [62]:
def boolean_grading (grade, pos_grades, neg_grades):
    if grade in pos_grades:
        return True
    elif grade in neg_grades:
        return False
    else:
        raise ValueError

# Überprüfung auf Bestanden/Durchgefallen
passing_grades = [1,2,3,4]
failing_grades = [5,6]
passed_s_int = s_int.apply(boolean_grading,args=(passing_grades,failing_grades))
print(passed_s_int)

s_passed_count = passed_s_int.value_counts()
s_passed_count.index = ["passed","failed"]
print(s_passed_count)


# Überprüfung auf 1 und 2
good_and_great_grades = [1,2]
other_grades = [3,4,5,6]
good_and_great_s_int = s_int.apply(
    boolean_grading,
    args=(good_and_great_grades,other_grades)
)
print(good_and_great_s_int)

s_good_great_count = good_and_great_s_int.value_counts()
s_good_great_count.index = ["1 or 2", "other"]
print(s_good_great_count)
!!!replace mit aufnehmen

0     True
1    False
2    False
3     True
4     True
5     True
6     True
7    False
8     True
9     True
dtype: bool
passed    7
failed    3
dtype: int64
0     True
1    False
2    False
3    False
4     True
5     True
6    False
7    False
8     True
9     True
dtype: bool
1 or 2    5
other     5
dtype: int64


### Konzept 2 DataFrame

In [1]:
# Die wichtigesten Pakete werden erneut importiert, um das Kapitel für sich betrachten zu können
# die Abkürzungen am besten beibehalten
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



Wie bereits erwähnt stellt eine Serie eine (Zeit-)Reihe dar, und besteht (neben vielen anderen Komponenten wie Datentyp und Name) aus einem Index und der Werte. Sollen mehrere Serien miteinander verbunden werden ist das DataFrame sehr nützlich. 

Im Gegensatz zu einer Serie welche - abgesehen vom Index - nur eine Spalte mit Werten besitzt, kann ein DataFrame beliebig viele Spalten in Tabellenform speichern.
![DataFrame_image](https://www.geeksforgeeks.org/wp-content/uploads/creating_dataframe1.png)

In der folgenden Code-Zelle wird zuerst die Erstellung und der Zugriff von Dataframes betrachtet. Wir empfehlen - ähnlich wie den Serien - jedes DataFrame mit `df` zu beginnen.  teilt sich viele Eigenschaften und Methoden mit den Serien und kann durch 
Das DataFrame kann auch als Kombination mehrerer Serien verstanden werden, welche die selbe Anzahl besitzen und nebeneinander "geklebt" wurden. Am besten ist es daher mit einer Tabelle vergleichbar, wobei auch jede Spalte als Serie gesehen kann.nur ein  erweitert werden, erhälttDas Dataframe stellt eine Tabellarische Darstellung von Daten zur Verfügung und erlaubt die Bearbeitung und Auswertung dieser. 

In [2]:
# Erstellung eines DataFrame aus einem Dictionairy
fruit_dict = {
    "Name": ["Apple", "Pears", "iuate", "itaer"],
    "Numbers": [10, 14, 16,100],
    "Cost": [2.0, 3.45, 5.12, 1.12]  # in Euro
}
df_fruit = pd.DataFrame(fruit_dict)


# Erstellung eines DataFrame durch hinzufügen von Serien
s_int = pd.Series([1,5,6,3,2,1,4,5,2,1])
# Eine Serie mit 10 Vornamen (alles klein geschrieben)
s_string = pd.Series(["alex", "paul", "paula","erika", "jan","karl", "vlad", "franziska", "sabrina", "florian"])
s_float = pd.Series(np.random.normal(173, 10, 10))
df_grades = pd.DataFrame()
df_grades["name"] = s_string
df_grades["grade"] = s_int
df_grades["size"] = s_float

# Erstellung eines DataFrame aus einer CSV-Datei
df_football = pd.read_csv("W1_csv-file.csv")
display(df_football)
print(df_grades)
print(df_fruit)

# Zugriff auf eine Spalte/Serie 
print(df_fruit["Name"])
# Unterschied zwischen print(df_grades) und df_grades als letzter Aufruf einer Zelle beachten
print(df_grades["grade"])
df_fruit

# !!! auch den Zugriff auf mehrehre Spalten mit angeben

,Name,Größe,Alter,Skill
0,Alex,179.45,19,9.6
1,Thorsten,178.43,23,8.9
2,Paul,178.23,21,9.7


        name  grade        size
0       alex      1  163.785146
1       paul      5  181.705791
2      paula      6  171.132242
3      erika      3  173.359460
4        jan      2  171.954352
5       karl      1  183.212112
6       vlad      4  178.496513
7  franziska      5  181.754546
8    sabrina      2  148.497315
9    florian      1  157.166278
    Name  Numbers  Cost
0  Apple       10  2.00
1  Pears       14  3.45
2  iuate       16  5.12
3  itaer      100  1.12
0    Apple
1    Pears
2    iuate
3    itaer
Name: Name, dtype: object
0    1
1    5
2    6
3    3
4    2
5    1
6    4
7    5
8    2
9    1
Name: grade, dtype: int64


,Name,Numbers,Cost
0,Apple,10,2.00
1,Pears,14,3.45
2,iuate,16,5.12
3,itaer,100,1.12


# Eigenschaften und Methoden 
Ähnlich wie Serien besitzen auch auch DataFrames Eigenschaften und Methoden. Auch bei dem DataFrame kann auch diese mittels `.` (Punkt) zugreifen. Hier einige Beispiele für Eigenschaften und Methoden, die sich DataFrame und Serie teilen...

In [33]:
# Einige Eigenschaften und Methoden haben DataFrame's und Serien gemeinsam
print("Einige Eigenschaften und Methoden haben DataFrames und Serien gemeinsman")
print(
    df_grades.index,
    df_grades.size,
    df_grades.shape,
    df_grades.count(),
    df_grades.isna(),
    sep="\n\n"
)

Einige Eigenschaften und Methoden haben DataFrames und Serien gemeinsman
RangeIndex(start=0, stop=10, step=1)

30

(10, 3)

name     10
grade    10
size     10
dtype: int64

    name  grade   size
0  False  False  False
1  False  False  False
2  False  False  False
3  False  False  False
4  False  False  False
5  False  False  False
6  False  False  False
7  False  False  False
8  False  False  False
9  False  False  False


Es gibt jedoch auch Eigenschaften und Methoden die nur das DataFrame besitzt. Beispielsweise zeigt die Eigenschaft `columns` die Spaltennamen an. Im folgenden gibt es einige Aufgaben, deren Lösung wichtige Methoden eines DataFrame benötigen... Oftmals ist bereits ein Hinweis gegeben, welche Methode benötigt wird nutze die Dokumentation (und das Internet) um die Aufgaben zu lösen

In [ ]:
Es gibt einige Methoden, die grundsätzliche Aussagen über ein DataFrame liefern. Dazu gehören 

In [34]:
#print(df_grades.dtypes)
# Andere Eigenschaften und Methoden kommen zusätzlich hinzu
print("\n\nAndere Eigenschaften und Methoden kommen zusätzlich hinzu:\n")
print(
    df_grades.columns,
    df_grades.query("grade <= 2"),
    sep="\n\n",
)
s_test=df_grades["grade"] <= 2
s_test



Andere Eigenschaften und Methoden kommen zusätzlich hinzu:

Index([&#39;name&#39;, &#39;grade&#39;, &#39;size&#39;], dtype=&#39;object&#39;)

      name  grade        size
0     alex      1  163.785146
4      jan      2  171.954352
5     karl      1  183.212112
8  sabrina      2  148.497315
9  florian      1  157.166278


0     True
1    False
2    False
3    False
4     True
5     True
6    False
7    False
8     True
9     True
Name: grade, dtype: bool

In [36]:
### W1A4 Umgang mit DataFrame
# Löse die folgenden Aufgaben
# gib für `df_grades` alle Einträge aus, die eine bessere Note als 3 haben (query)
# df_grades.query(...)
# zeige in `df_football` lediglich die beiden Spalten `name` und `skill` an (filter)
# Ändere die Spaltenüberschriften für `df_fruit` in die deutschen Übersetzungen
# ändere den ersten Buchstaben aller Namen in der Spalte `name` in `df_grades` in Großbuchstaben
# bestimme den Mittelwert und die Standardabweichung der Spalte `size` von `df_grades`
# Zeige an, wie die Verteilung an Noten in `df_grades` ist (groupby)
df_grades.groupby("grade").count()
# Sortiere das DataFrame `df_grades` absteigend nach der Größe (sort_values)
#  !!! visualisierung
# plt.figure() wenn eine neue Darstellung erstellt werden soll
# df.hist() oder so für darstellung (geht auch mit serien)
# kann ziemlich kompliziert werden am besten einfach halten und plot.hist() verwenden
# wird plt.figure() nicht verwendet werden Alle Befehle zusammengesteckt

,name,size
grade,,
1,3,3
2,2,2
3,1,1
4,1,1
5,2,2
6,1,1
